<a href="https://colab.research.google.com/github/lorransr/ice_fire_and_data/blob/master/Uma_can%C3%A7%C3%A3o_de_Gelo%2C_Fogo_e_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uma canção de Fogo, Gelo e Dados

In [3]:
pip install chartify

In [0]:
#importando as libs
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from pandas.io.json import json_normalize
import json
import chartify
#fazer funcionar o bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

In [6]:
!git clone -l -s git://github.com/lorransr/ice_fire_and_data.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (12/12), 191.77 KiB | 1022.00 KiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/cloned-repo
episodes.csv  episodes.json  README.md	Uma_canção_de_Gelo,_Fogo_e_Dados.ipynb


In [0]:
with open('../cloned-repo/episodes.json') as f:
    d = json.load(f)

recapitulando: 
* Puxei os dados do Json "episodes"; 
* Segmentei por cenas e personagens;
* mantive os campos meta referentes ao episódio;
* mantive o inicio e o fim da cena;

In [8]:

#https://www.kaggle.com/jboysen/quick-tutorial-flatten-nested-json-in-pandas
#trabalhando com o json
raw = json_normalize(data = d['episodes'],
                     record_path = ["scenes","characters"],
                     meta=["seasonNum","episodeNum",["scenes","sceneStart"],["scenes","sceneEnd"],"episodeAirDate"],
                     sep = "," ,
                     errors = "ignore")
raw.head(10)

,alive,born,killedBy,mannerOfDeath,married,name,sex,title,weapon,seasonNum,episodeNum,"scenes,sceneStart","scenes,sceneEnd",episodeAirDate
0,NaN,NaN,NaN,NaN,NaN,Gared,NaN,NaN,NaN,1,1,0:00:40,0:01:45,2011-04-17
1,NaN,NaN,NaN,NaN,NaN,Waymar Royce,NaN,NaN,NaN,1,1,0:00:40,0:01:45,2011-04-17
2,NaN,NaN,NaN,NaN,NaN,Will,NaN,NaN,NaN,1,1,0:00:40,0:01:45,2011-04-17
3,NaN,NaN,NaN,NaN,NaN,Gared,NaN,NaN,NaN,1,1,0:01:45,0:03:24,2011-04-17
4,NaN,NaN,NaN,NaN,NaN,Waymar Royce,NaN,NaN,NaN,1,1,0:01:45,0:03:24,2011-04-17
5,NaN,NaN,NaN,NaN,NaN,Will,NaN,NaN,NaN,1,1,0:01:45,0:03:24,2011-04-17
6,NaN,NaN,NaN,NaN,NaN,Will,NaN,NaN,NaN,1,1,0:03:24,0:03:31,2011-04-17
7,False,NaN,NaN,NaN,NaN,Wight Wildling Girl,NaN,NaN,NaN,1,1,0:03:24,0:03:31,2011-04-17
8,NaN,NaN,NaN,NaN,NaN,Will,NaN,NaN,NaN,1,1,0:03:31,0:03:38,2011-04-17
9,NaN,NaN,NaN,NaN,NaN,Gared,NaN,NaN,NaN,1,1,0:03:44,0:05:36,2011-04-17


In [0]:
df = raw

In [10]:
df.columns

Index(['alive', 'born', 'killedBy', 'mannerOfDeath', 'married', 'name', 'sex',
       'title', 'weapon', 'seasonNum', 'episodeNum', 'scenes,sceneStart',
       'scenes,sceneEnd', 'episodeAirDate'],
      dtype='object')

In [11]:
df.columns=['alive', 'born', 'killedBy', 'mannerOfDeath', 'married', 'name', 'sex',
            'title', 'weapon', 'seasonNum', 'episodeNum', 'sceneStart',
            'sceneEnd', 'episodeAirDate']
df.dtypes

alive             object
born              object
killedBy          object
mannerOfDeath     object
married           object
name              object
sex               object
title             object
weapon            object
seasonNum          int64
episodeNum         int64
sceneStart        object
sceneEnd          object
episodeAirDate    object
dtype: object

In [0]:
df["sceneEnd_s"]=df["sceneEnd"].str.split(":")
df["sceneEnd_s"]=df["sceneEnd_s"].str[0].astype(int)*3600+df["sceneEnd_s"].str[1].astype(int)*60+df["sceneEnd_s"].str[2].astype(int)

df["sceneStart_s"]=df["sceneStart"].str.split(":")
df["sceneStart_s"]=df["sceneStart_s"].str[0].astype(int)*3600+df["sceneStart_s"].str[1].astype(int)*60+df["sceneStart_s"].str[2].astype(int)

df["screenTime"] = df["sceneEnd_s"]-df["sceneStart_s"]


In [13]:
df.head()

,alive,born,killedBy,mannerOfDeath,married,name,sex,title,weapon,seasonNum,episodeNum,sceneStart,sceneEnd,episodeAirDate,sceneEnd_s,sceneStart_s,screenTime
0,NaN,NaN,NaN,NaN,NaN,Gared,NaN,NaN,NaN,1,1,0:00:40,0:01:45,2011-04-17,105,40,65
1,NaN,NaN,NaN,NaN,NaN,Waymar Royce,NaN,NaN,NaN,1,1,0:00:40,0:01:45,2011-04-17,105,40,65
2,NaN,NaN,NaN,NaN,NaN,Will,NaN,NaN,NaN,1,1,0:00:40,0:01:45,2011-04-17,105,40,65
3,NaN,NaN,NaN,NaN,NaN,Gared,NaN,NaN,NaN,1,1,0:01:45,0:03:24,2011-04-17,204,105,99
4,NaN,NaN,NaN,NaN,NaN,Waymar Royce,NaN,NaN,NaN,1,1,0:01:45,0:03:24,2011-04-17,204,105,99


## Quem apareceu mais vezes em cena 

In [18]:
df["name2"] = df["name"]
rank = df.groupby(by = ["seasonNum","name"])[["name2"]].count().reset_index()
rank.columns = ["seasonNum","name","count"]
rank.sort_values(by=["seasonNum","count"],ascending=False,inplace=True)

rank.head()

,seasonNum,name,count
1018,8,Daenerys Targaryen,115
1032,8,Jon Snow,105
1054,8,Tyrion Lannister,98
1031,8,Jaime Lannister,68
1049,8,Sansa Stark,58


In [61]:
import altair as alt


bars = alt.Chart(rank).mark_bar().encode(
        y= alt.Y('name',sort=alt.EncodingSortField("count",order="descending")),
        x='count',
        color=alt.Color('seasonNum',scale=alt.Scale(scheme ="blueorange" )),
        order = alt.Order("seasonNum",sort="ascending"))
bars

Chart({
  data:       seasonNum                  name  count
  1018          8    Daenerys Targaryen    115
  1032          8              Jon Snow    105
  1054          8      Tyrion Lannister     98
  1031          8       Jaime Lannister     68
  1049          8           Sansa Stark     58
  1010          8            Arya Stark     57
  1014          8      Brienne of Tarth     57
  1019          8        Davos Seaworth     57
  1029          8             Grey Worm     55
  1013          8            Bran Stark     54
  1035          8            Lord Varys     54
  1047          8         Samwell Tarly     50
  1021          8                Drogon     49
  1053          8    Tormund Giantsbane     49
  1033          8         Jorah Mormont     47
  1048          8        Sandor Clegane     41
  1040          8             Missandei     39
  1042          8         Podrick Payne     36
  1045          8               Rhaegal     36
  1025          8                Gendry     35
  1012          8      Beric Dondarrion     27
  1036          8        Lyanna Mormont     25
  1039          8            Melisandre     22
  1051          8        The Night King     22
  1052          8         Theon Greyjoy     21
  1023          8         Euron Greyjoy     20
  1062          8            Yohn Royce     20
  1044          8                Qyburn     18
  1022          8       Eddison Tollett     17
  1056          8              Viserion     17
  ...         ...                   ...    ...
  121           1   Wight Wildling Girl      2
  0             1        Addam Marbrand      1
  2             1                Armeca      1
  6             1      Beric Dondarrion      1
  10            1      Catspaw Assassin      1
  15            1        Dothraki Crone      1
  27            1               Hot Pie      1
  32            1         Jafer Flowers      1
  35            1         Jaremy Rykker      1
  39            1             Jon Arryn      1
  41            1         Jonos Bracken      1
  44            1                  Joss      1
  45            1      Joyeuse Erenford      1
  48            1  King's Landing Baker      1
  51            1   Lannister Messenger      1
  53            1           Leo Lefford      1
  54            1           Little Bird      1
  55            1      Lommy Greenhands      1
  64            1               Mhaegen      1
  79            1    Red Keep Stableboy      1
  83            1      Rickard Karstark      1
  91            1          Ryger Rivers      1
  99            1          Stevron Frey      1
  100           1                  Stiv      1
  101           1         Street Urchin      1
  107           1            Tobho Mott      1
  108           1                Tomard      1
  113           1                 Varly      1
  114           1           Vayon Poole      1
  118           1                Wallen      1
  
  [1063 rows x 3 columns],
  encoding: FacetedEncoding({
    color: Color({
      scale: Scale({
        scheme: 'blueorange'
      }),
      shorthand: 'seasonNum'
    }),
    order: Order({
      shorthand: 'seasonNum',
      sort: 'ascending'
    }),
    x: X({
      shorthand: 'count'
    }),
    y: Y({
      shorthand: 'name',
      sort: EncodingSortField({
        field: 'count',
        order: 'descending'
      })
    })
  }),
  mark: 'bar'
})